Full Exercise Classes Example
-----------------------------

Below, we create a dispatcher that creates an exercise instance.
This is done when the `runInit` command is dispatched.


Exercises
---------

In [1]:
class BaseEx:
    def __init__(self, data):
        print('running __init__ for %s' %self.__class__.__name__)

    def runInit(self, data):
        self.x = data['x']
        return self.x
    
    def getX(self, data):
        return self.x

class SubEx(BaseEx):
    def add(self, n):
        self.x += n

Basic Exercise Example
----------------------

In [2]:
from protobackend.dispatcher import Dispatcher

d = Dispatcher({'BaseEx': BaseEx, 'SubEx': SubEx})

@d.hook('pre')
def init_class(data, cmd, dispatcher):
    """Create a class instance. Assigned to d.active_exercise."""
    if cmd == 'runInit':
        # get class name from payload
        name = data.get('class', 'BaseEx')
        cls = dispatcher.exercises[name]
        print('pre hook class: ', cls.__name__)
        dispatcher.active_exercise = cls(data)
    return data

@d.hook('post')
def print_output(data, cmd):
    print('%s:\t'%cmd, data)

print(d.exercises)
print(d.active_exercise)

{'SubEx': <class '__main__.SubEx'>, 'BaseEx': <class '__main__.BaseEx'>}
None


In [3]:
print("Running BaseEx ----")
d.dispatch({
        'command': 'runInit',
        'payload': {'class': 'BaseEx', 'x': 1}
    })
d.dispatch({'command': 'getX', 'payload': None})

print(d.active_exercise)

Running BaseEx ----
pre hook class:  BaseEx
running __init__ for BaseEx
runInit:	 1
getX:	 1


In [4]:
print("Running SubEx ----")
d.dispatch({
        'command': 'runInit',
        'payload': {'class': 'SubEx', 'x': 2}
    })
d.dispatch({'command': 'add', 'payload': 10})
d.dispatch({'command': 'getX', 'payload': None})

print(d.active_exercise)

Running SubEx ----
pre hook class:  SubEx
running __init__ for SubEx
runInit:	 2
add:	 None
getX:	 12


Exposing Commands for Multiplexer
------------------------------------

In [5]:
# init_hook is analogous to the function init_class
# that was written manually in the demo above.
# it expects the DC_TYPE to specify the class name
from protobackend.dispatcher import init_hook

d = Dispatcher({'BaseEx': BaseEx, 'SubEx': SubEx})

# how python decorator syntax looks without @
d.hook('pre')(init_hook)

# calling runInit(payload) will be equivalent to:
# d.dispatch({'command': 'runInit', 'payload': payload})
runInit = d.expose('runInit')
getX = d.expose('getX')

runInit({'DC_TYPE': 'SubEx', 'x': 2})
getX({})

running __init__ for SubEx


2

Communicating with Multiplexer
------------------------------

The multiplexer is cruel. It expects a json object, that is then converted to json again.

In [6]:
from protobackend.output import print_output, output_dec, safe_dump
import json

print_output(json.dumps(["cool"]))


[1] "[\"cool\"]"

>>> 


In [7]:
@safe_dump
def unsafe_func():
    raise Exception("uh-oh")

print_output(unsafe_func())


[1] "[{\"payload\": \"DataCamp encountered the following error:\\nuh-oh\\n\", \"type\": \"backend-error\"}]"

>>> 


In [8]:
@output_dec
@safe_dump
def full_monty(x):
    return x + 1

full_monty(1)    # prints 2
full_monty()     # prints error payload
None


[1] "2"

>>> 

[1] "[{\"payload\": \"DataCamp encountered the following error:\\nfull_monty() missing 1 required positional argument: 'x'\\n\", \"type\": \"backend-error\"}]"

>>> 
